In [ ]:
import pandas as pd

df = pd.read_csv('/content/combined_newcsv.csv', skipinitialspace=True)
df = df.drop(['Unnamed: 0'], axis=1)
df

,Review Rate,Review Text
0,5 stars,Was here for lunch. Had ordered a pork BBQ set...
1,5 stars,I will give 10 stars if permitted! Food is gre...
2,5 stars,I just keep returning. ❤️🌟This is absolutely m...
3,5 stars,"Excellent service, reasonably priced set meal ..."
4,5 stars,Throwback to one of the better Korean BBQs I h...
...,...,...
177514,5 stars,(Translated by Google) The sea breeze blows co...
177515,4 stars,"(Translated by Google) Not bad, very comfortab..."
177516,5 stars,One of the earliest specialize in White Bee Ho...
177517,5 stars,White beehoon is the signature. You don't eat ...


In [ ]:
df['Labels'] = df['Review Rate'].str.replace('star', '')
df['Labels'] = df['Labels'].str.replace('s', '')
df['Labels'] = df['Labels'].str.strip()
df['Labels'] = df['Labels'].astype(int)
df['Labels'] = df['Labels'].map(lambda x: x-1)
df = df.drop(["Review Rate"], axis=1)
df['Review Text'] = df["Review Text"].str.split('\(Original\)').str[0]
df['Review Text'] = df["Review Text"].str.replace('\n', ' ')
df['Review Text'] = df["Review Text"].str.replace('\(Translated by Google\)', '')
df['Review Text'] = df["Review Text"].str.strip()
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


,Review Text,Labels
0,Was here for lunch. Had ordered a pork BBQ set...,4
1,I will give 10 stars if permitted! Food is gre...,4
2,I just keep returning. ❤️🌟This is absolutely m...,4
3,"Excellent service, reasonably priced set meal ...",4
4,Throwback to one of the better Korean BBQs I h...,4
...,...,...
177514,"The sea breeze blows cool, the park pavilion c...",4
177515,"Not bad, very comfortable environment, the foo...",3
177516,One of the earliest specialize in White Bee Ho...,4
177517,White beehoon is the signature. You don't eat ...,4


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

y = tf.keras.utils.to_categorical(df["Labels"].values, num_classes=5)

x_train, x_Test, y_train, y_Test = train_test_split(df['Review Text'], y, test_size=0.12)

x_val, x_test, y_val, y_test = train_test_split(x_Test, y_Test, test_size=0.5)

In [ ]:
y_val.shape

(10651, 5)

In [ ]:
x_val


{'input_ids': <tf.Tensor: shape=(10651, 30), dtype=int32, numpy=
array([[  101,  6702,  1110, ...,  1106,  2867,   102],
       [  101,   138,  3014, ...,  1122,  1106,   102],
       [  101,  6424,  1363, ...,     0,     0,     0],
       ...,
       [  101, 11336,  6598, ...,     0,     0,     0],
       [  101, 23878,  1303, ...,  1186,   117,   102],
       [  101, 20394,  4490, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(10651, 30), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
!pip install transformers
import transformers
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.2 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 59.8 MB/s 
     |████████████████████████████████| 6.6 MB 55.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=x_train.values.tolist(),
    add_special_tokens=True,
    max_length=30,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_val = tokenizer(
    text=x_val.values.tolist(),
    add_special_tokens=True,
    max_length=30,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=x_test.values.tolist(),
    add_special_tokens=True,
    max_length=30,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv1D, MaxPool1D, Embedding, Concatenate, Dropout,GlobalMaxPool1D,Lambda
from tensorflow.keras.layers import Dense, Input, Flatten,SpatialDropout1D

In [ ]:
import math
max_len = 30
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
out = bert(input_ids,attention_mask = input_mask)[0]
out = Dropout(0.2)(out) 
filter_lengths = [2, 3, 4, 5]
conv_layers = []
for filter_length in filter_lengths:
    conv_layer = Conv1D(filters=8*math.pow(2, filter_length), kernel_size=filter_length, padding='valid',
                        strides=1, activation='relu')(out)
    maxpooling = MaxPool1D(pool_size=max_len - filter_length + 1)(conv_layer)
    maxpooling = Dropout(0.4)(maxpooling)
    flatten = Flatten()(maxpooling)
    conv_layers.append(flatten)
out = Concatenate()(conv_layers)
out = Dense(128, activation='relu')(out)
out = Dropout(0.2)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(5,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
print([input_ids, input_mask].shape)
model.layers[2].trainable = True

AttributeError: ignored

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 30)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 30,                                          

In [ ]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.001,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 2,
                                                      restore_best_weights = True)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']}, y_val
    ),
  epochs=5,
  batch_size=8,
  callbacks = [earlystop_callback]
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


19527/19527 [==============================] - 1807s 92ms/step - loss: 0.7977 - balanced_accuracy: 0.6581 - val_loss: 0.7998 - val_balanced_accuracy: 0.6841
Epoch 2/5
19527/19527 [==============================] - 1784s 91ms/step - loss: 0.7115 - balanced_accuracy: 0.6991 - val_loss: 0.7772 - val_balanced_accuracy: 0.6856
Epoch 3/5
19527/19527 [==============================] - 1778s 91ms/step - loss: 0.6828 - balanced_accuracy: 0.7141 - val_loss: 0.7807 - val_balanced_accuracy: 0.6884
Epoch 4/5
19527/19527 [==============================] - 1778s 91ms/step - loss: 0.6682 - balanced_accuracy: 0.7210 - val_loss: 0.7836 - val_balanced_accuracy: 0.6849


In [ ]:
model.save("/models/text_classifier_v1")

INFO:tensorflow:Assets written to: /models/text_classifier_v1/assets


INFO:tensorflow:Assets written to: /models/text_classifier_v1/assets


In [ ]:

!zip -r /content/model.zip /models/text_classifier_v1

  adding: models/text_classifier_v1/ (stored 0%)
  adding: models/text_classifier_v1/variables/ (stored 0%)
  adding: models/text_classifier_v1/variables/variables.data-00000-of-00001 (deflated 15%)
  adding: models/text_classifier_v1/variables/variables.index (deflated 80%)
  adding: models/text_classifier_v1/keras_metadata.pb (deflated 95%)
  adding: models/text_classifier_v1/saved_model.pb (deflated 92%)
  adding: models/text_classifier_v1/assets/ (stored 0%)


In [ ]:
from google.colab import files
files.download('/models/text_classifier_v1')

In [ ]:

new_model = tf.saved_model.load('/models/text_classifier_v1')


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

model = tf.keras.Sequential([
    hub.KerasLayer("/models/text_classifier_v1")
])
model.build(input_shape=(2,))

ValueError: ignored

In [ ]:
predicted_raw = new_model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

AttributeError: ignored